In [1]:
import torch
import json
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import config
import nltk
from sentence_transformers import SentenceTransformer
import pymongo
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import MongoDBAtlasVectorSearch

In [2]:
with open("swagger.json",'r') as f:
    swagger = f.read()

#print(swagger)

In [3]:
swagger = json.loads(swagger)

In [4]:
swagger['basePath'] = "https://api.corestack.io"

In [5]:
li = []
for path in swagger['paths']:
    li.append(path)
#li

## Create new dict with the derefenced-api path

In [6]:
new_dict = {}

for path in li:
    new_dict['swagger'] = swagger['swagger']
    new_dict['basePath']= swagger['basePath']
    new_dict[swagger['basePath']+path] = swagger['paths'][path]
    for method in swagger['paths'][path]:
        if 'responses' in swagger['paths'][path][method]:
            for response in swagger['paths'][path][method]['responses']:
                if 'schema' in swagger['paths'][path][method]['responses'][response]:
                    if '$ref' in swagger['paths'][path][method]['responses'][response]['schema']:
                        ref = swagger['paths'][path][method]['responses'][response]['schema']['$ref']
                        ref = swagger['basePath']+path+ref[1:]
                        new_dict[swagger['basePath']+path][method]['responses'][response]['schema']['ref']  = ref


#new_dict

# save new json 

In [7]:
#with open("swagger-with-path.json", "w") as outfile: 
#    json.dump(new_dict, outfile)

### Using Model here to create a RAG pipeline
- Moved to ollama as llama.cpp is not working for my usecase. 

In [8]:
#llm = Ollama(model="openhermes", request_timeout=30.0)

#response = llm.complete("What is the history of LEGO?")
#print(response)

In [9]:
# messages = [
#     ChatMessage(
#         role="system", content="You are a pirate with a colorful personality"
#     ),
#     ChatMessage(role="user", content="What is your name"),
# ]
# resp = llm.chat(messages)
# print(resp)

In [10]:
#langchain implementation
from  langchain_community.document_loaders import JSONLoader

loader = JSONLoader(file_path="swagger-derefernced.json",jq_schema=".",text_content=False)

docs = loader.load()


In [11]:
docs

[Document(page_content='{"swagger": "2.0", "basePath": "https://api.corestack.io", "https://api.corestack.io/access/posture/governance_summary/list": {"get": {"responses": {"500": {"description": "Internal Server Error", "schema": {"$ref": "#/definitions/ModelError", "ref": "https://api.corestack.io/definitions/ModelError"}}, "401": {"description": "Unauthorized", "schema": {"$ref": "#/definitions/ModelError", "ref": "https://api.corestack.io/definitions/ModelError"}}, "400": {"description": "Bad Request", "schema": {"$ref": "#/definitions/ModelError", "ref": "https://api.corestack.io/definitions/ModelError"}}, "200": {"description": "Success", "schema": {"$ref": "#/definitions/SummaryCountResponse", "ref": "https://api.corestack.io/definitions/SummaryCountResponse"}}}, "summary": "Get Access Summary Count", "description": "Displays the summary of the violations based on the cloud provider", "operationId": "AccessSummaryCount", "parameters": [{"description": "Supported services for Acc

In [12]:

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)
len(chunks)

1

In [13]:
#Load MongoDB
mongo = config.mongodb_uri
dbname = "swagger"
collection = "embd"


In [14]:
client = pymongo.MongoClient(mongo)
db = client[dbname]
coll = db[collection]

coll

Collection(Database(MongoClient(host=['ac-rnd695q-shard-00-01.yq6vcdv.mongodb.net:27017', 'ac-rnd695q-shard-00-00.yq6vcdv.mongodb.net:27017', 'ac-rnd695q-shard-00-02.yq6vcdv.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-golhgl-shard-0', tls=True), 'swagger'), 'embd')

In [15]:

vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="openhermes"),
    collection=coll,
    index_name="default"  # Use a predefined index name
)

In [16]:
from langchain_community.llms import Ollama
llm = Ollama(model="openhermes")


In [21]:
query = "How to create a tenant using the API in corestack?"

In [22]:
retriever = vector_search.as_retriever()  

In [23]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
                llm, chain_type="stuff", retriever=retriever
            )
response = qa({"query": query})

response

{'query': 'How to create a tenant using the API in corestack?',
 'result': 'To create a tenant using the API in CoreStack, you need to follow these steps:\n\n1. Obtain the necessary credentials and access tokens required for API authentication.\n2. Set up the appropriate headers and parameters for your API request, including any required data like tenant details (name, email, etc.).\n3. Make a POST request to the CoreStack API endpoint responsible for creating tenants.\n4. Monitor the response for any error messages or success codes, and handle them accordingly in your code or application.\n5. Process the returned data, if any, which may include details of the newly created tenant or other relevant information.\n\nHere\'s an example using cURL:\n```bash\ncurl -X POST \\\n  \'https://api.corestack.io/v1alpha1/tenants\' \\\n  -H \'Authorization: Bearer YOUR_ACCESS_TOKEN\' \\\n  -H \'Content-Type: application/json\' \\\n  -d \'{\n      "name": "example-tenant",\n      "email": "example@ex

In [24]:
#failed usecase, model is genrating trash response

In [25]:
#using Llama index